In [ ]:
import tensorflow as tf
import numpy as np
import os
import math
file_dir = '/home/weifeng/learngit/tf/cats vs dogs/train/'

# 将照片写成list

In [1]:
def get_file(file_dir,ratio):
    cats = []
    label_cats = []
    dogs = []
    label_dogs = []
    for file in os.listdir(file_dir):
        name = file.split(sep='.')
        if name[0] == 'cat':
            cats.append(file_dir + file)
            label_cats.append(0)
            
        else:
            dogs.append(file_dir + file)
            label_dogs.append(1)
            
    print('There are %d cats \n There are %d dogs' %(len(cats),len(dogs)))
    image_list = np.hstack((cats,dogs))
    label_list = np.hstack((label_cats,label_dogs))
    
    temp =  np.array([image_list,label_list])
    temp = temp.T
    np.random.shuffle(temp)
    
    all_image_list = temp[:,0]
    all_label_list = temp[:,1]
    
    n_sample = len(all_image_list)
    n_val = math.ceil(n_sample * ratio)
    n_train = n_sample - n_val
    
    train_image = all_image_list[0:n_train]
    train_label = all_label_list[0:n_train]
    train_label = [int(float(i))for i in train_label]
    
    val_image = all_image_list[n_train:]
    val_label = all_label_list[n_train:]
    val_label = [int(float(i))for i in val_label]
    
    return train_image,train_label,val_image,val_label
    
    
    
    
    

In [2]:
def get_batch(image,label,image_W,image_H,batch_size):
    image = tf.cast(image,tf.string)
    label = tf.cast(label,tf.int32)
    
    input_queue = tf.train.slice_input_producer([image,label])
    label = input_queue[1]
    image_contents = tf.read_file(input_queue[0])
    image = tf.image.decode_jpeg(image_contents, channels=3)

    image = tf.image.resize_image_with_crop_or_pad(image, image_W, image_H)

    # if you want to test the generated batches of images, you might want to comment the following line.
    image = tf.image.per_image_standardization(image)
    image_batch, label_batch = tf.train.batch([image, label],
                                                batch_size= batch_size,
                                                num_threads= 64
                                                )
    label_batch = tf.reshape(label_batch, [batch_size])
    image_batch = tf.cast(image_batch, tf.float32)

    return image_batch, label_batch

 

In [ ]:
def inference(images,batch_size,n_nclass):
    with tf.variable_scope('cov1') as scope:
        weights = tf.get_variable('weights',shape = [])